In [6]:
from astropy.io import fits
import math
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib
import random
import scipy.stats as stats
import scipy.special as special
import emcee
import corner
import math


hdu=fits.open( 'outerdisk_sample.fits' )
hdu.info()
print(hdu[1].header)
r=hdu[1].data.field('rgc')
rv=hdu[1].data.field('rv')
rv_err=hdu[1].data.field('rv_err')
nansr = np.isnan(r)
rv = np.delete(rv, np.where(nansr))
rv_err= np.delete(rv_err, np.where(nansr))
r= np.delete(r, np.where(nansr))
m=np.mean(rv)
print(m)

def wudibaolongzhansheng(x,rv,rv_err,r):
    a=x[0]
    hr=x[1]
    dr = abs((r - 8.34) * 0.2)
    r0=stats.norm.rvs(loc=r, scale=dr, size=len(r))
    sig2=a*np.exp(-r0/hr)
    sig2z=abs(sig2+(rv_err**2))
    L = np.sum(-((rv-6.17)**2)/(2*sig2z)-0.5*np.log(sig2z))
    return L



Filename: outerdisk_sample.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (10006,)   uint8   
  1  Joined        1 BinTableHDU     75   23983R x 18C   [E, E, E, E, E, E, D, D, D, D, D, D, D, D, D, D, D, L]   
XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / 8-bit bytes                                    NAXIS   =                    2 / 2-dimensional table                            NAXIS1  =                  113 / width of table in bytes                        NAXIS2  =                23983 / number of rows in table                        PCOUNT  =                    0 / size of special data area                      GCOUNT  =                    1 / one data group                                 TFIELDS =                   18 / number of columns                              EXTNAME = 'Joined  '           / table name                                     TTYPE1  =

In [5]:
nwalker = 30
ndim = 2
p0 = np.zeros((nwalker,ndim))
pos = np.random.randn(nwalker, ndim)
p0[:,0] = np.random.rand(nwalker)*100+1500
p0[:,1] = np.random.rand(nwalker)+15

sampler = emcee.EnsembleSampler(nwalker, ndim, wudibaolongzhansheng,args=[rv_err,r])
pos = sampler.run_mcmc(p0,100)
sampler.reset()
sampler.run_mcmc(pos,1000)
samples = sampler.chain[:, :, :].reshape((-1, ndim))
fig = corner.corner(samples,\
    labels=[r'$d$',r'$d_0$'],\
        quantiles=[0.16, 0.5, 0.84],\
        show_titles=True, \
        title_kwargs={"fontsize": 12})

emcee: Exception while calling your likelihood function:
  params: [1577.56100202   15.9280537 ]
  args: [array([15.41, 12.  , 17.71, ..., 15.1 , 14.99, 11.84], dtype='>f4')]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/sralus/.conda/envs/astrostatistics/lib/python3.11/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: wudibaolongzhansheng() missing 2 required positional arguments: 'rv_err' and 'r'


TypeError: wudibaolongzhansheng() missing 2 required positional arguments: 'rv_err' and 'r'